## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leningradskiy,RU,69.3833,178.4167,37.24,93,100,5.19,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,71.65,83,13,4.61,few clouds
2,2,Hithadhoo,MV,-0.6000,73.0833,82.51,77,24,11.70,few clouds
3,3,Vaini,TO,-21.2000,-175.2000,71.76,94,20,2.30,fog
4,4,Chokurdakh,RU,70.6333,147.9167,39.02,82,63,14.65,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation "))
max_temp = float(input("What is the maximum temperature you would like for your vacation "))


What is the minimum temperature you would like for your vacation 60
What is the maximum temperature you would like for your vacation 61


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
156,156,Mizan Teferi,ET,6.9833,35.5833,60.89,98,98,1.59,light rain
233,233,Butembo,CD,0.1500,29.2833,60.26,84,97,2.57,overcast clouds
245,245,Bonavista,CA,48.6499,-53.1147,60.08,77,50,6.44,scattered clouds
347,347,Zonguldak,TR,41.2500,31.8333,60.48,87,0,1.14,clear sky
378,378,Haines Junction,CA,60.7522,-137.5108,60.73,49,62,5.93,broken clouds
384,384,Port Hardy,CA,50.6996,-127.4199,60.98,94,90,3.44,fog
417,417,Hervey Bay,AU,-25.2986,152.8535,60.04,54,1,11.21,clear sky
449,449,Mporokoso,ZM,-9.3727,30.1250,60.51,63,3,13.53,clear sky
462,462,College,US,64.8569,-147.8028,60.75,92,75,8.05,broken clouds
475,475,Pori,FI,61.4833,21.7833,60.87,77,0,4.61,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                10
City                   10
Country                10
Lat                    10
Lng                    10
Max Temp               10
Humidity               10
Cloudiness             10
Wind Speed             10
Current Description    10
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df.count()

City_ID                10
City                   10
Country                10
Lat                    10
Lng                    10
Max Temp               10
Humidity               10
Cloudiness             10
Wind Speed             10
Current Description    10
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
156,Mizan Teferi,ET,60.89,light rain,6.9833,35.5833,
233,Butembo,CD,60.26,overcast clouds,0.1500,29.2833,
245,Bonavista,CA,60.08,scattered clouds,48.6499,-53.1147,
347,Zonguldak,TR,60.48,clear sky,41.2500,31.8333,
378,Haines Junction,CA,60.73,broken clouds,60.7522,-137.5108,
384,Port Hardy,CA,60.98,fog,50.6996,-127.4199,
417,Hervey Bay,AU,60.04,clear sky,-25.2986,152.8535,
449,Mporokoso,ZM,60.51,clear sky,-9.3727,30.1250,
462,College,US,60.75,broken clouds,64.8569,-147.8028,
475,Pori,FI,60.87,clear sky,61.4833,21.7833,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
156,Mizan Teferi,ET,60.89,light rain,6.9833,35.5833,Hotel Salayish | ሆቴል ሳላይሽ
233,Butembo,CD,60.26,overcast clouds,0.1500,29.2833,Hotel Butembo
245,Bonavista,CA,60.08,scattered clouds,48.6499,-53.1147,Abbott's Bed & Breakfast
347,Zonguldak,TR,60.48,clear sky,41.2500,31.8333,Süleyman Tez Ev
378,Haines Junction,CA,60.73,broken clouds,60.7522,-137.5108,Parkside Inn


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label= "City_ID")

In [18]:
vacation_df = pd.read_csv("./weather_data/WeatherPy_Vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,156,Mizan Teferi,ET,60.89,light rain,6.9833,35.5833,Hotel Salayish | ሆቴል ሳላይሽ
1,233,Butembo,CD,60.26,overcast clouds,0.1500,29.2833,Hotel Butembo
2,245,Bonavista,CA,60.08,scattered clouds,48.6499,-53.1147,Abbott's Bed & Breakfast
3,347,Zonguldak,TR,60.48,clear sky,41.2500,31.8333,Süleyman Tez Ev
4,378,Haines Junction,CA,60.73,broken clouds,60.7522,-137.5108,Parkside Inn


In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd> at{Max Temp}</dd> 
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer) 
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))